In [ ]:
#Classes 
from OnlineDetectors import FilterAlgorithm, NetworkModels, DataSetManipulator

import tensorflow_datasets as tfds

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch import optim 

import numpy as np

device = torch.device("cuda:0")

torch.cuda.set_device(device)

# Import and set-up MNIST data set from tensorflow datasets 
Here we set up the data sets for training the neural network

In [ ]:
training_images, training_labels = tfds.as_numpy(tfds.load(
                                        'mnist',
                                        split='train', 
                                        batch_size=-1, 
                                        as_supervised=True,
                                    ))

testing_images, testing_labels = tfds.as_numpy(tfds.load(
                                        'mnist',
                                        split='test', 
                                        batch_size=-1, 
                                        as_supervised=True,
                                    ))

In [ ]:
model = nn.Sequential(
                nn.Conv1d(1,10,kernel_size=(3,3)),
                nn.ReLU(),
                nn.Conv1d(10,100,kernel_size=(3,3)),
                nn.ReLU(),
                nn.Flatten(),
                nn.Linear(57600,10),
            )

In [ ]:
training_images_pytorch = torch.Tensor(training_images).transpose(1,3)
training_labels_pytorch = torch.Tensor(training_labels).type(torch.LongTensor)

testing_images_pytorch = torch.Tensor(testing_images).transpose(1,3)
testing_labels_pytorch = torch.Tensor(testing_labels).type(torch.LongTensor)

training_dataset = TensorDataset(training_images_pytorch,training_labels_pytorch)
testing_dataset  = TensorDataset(testing_images_pytorch,testing_labels_pytorch)

training_loader = DataLoader(training_dataset, batch_size=5, shuffle=True, pin_memory=True,drop_last=True)
testing_loader  = DataLoader(testing_dataset, batch_size=5, shuffle=True, pin_memory=True,drop_last=True)

In [ ]:
loss_array, accuracy_array = NetworkModels.TrainNetwork(model,device,training_loader,testing_loader,10)

# Find frequency of labels within training set
The function `FilterAlgorithm.calculate_lambda` calculates the frequency a label appears in the training data set. Then, the function `FilterAlgorithm.runDetector` detects if there is a change in label distribution 

In [ ]:
lambda_vals_training = FilterAlgorithm.calculate_lambda(training_labels_pytorch,torch.Tensor(np.arange(0,10)).int())

In [ ]:
testing_images_pytorch = torch.Tensor(testing_images).transpose(1,3)
testing_labels_pytorch = torch.Tensor(testing_labels).type(torch.LongTensor)
testing_dataset        = TensorDataset(testing_images_pytorch,testing_labels_pytorch)
testing_loader         = DataLoader(testing_dataset, batch_size=1, shuffle=True, pin_memory=True,drop_last=True)

detection_images,detection_labels = DataSetManipulator.shiftLabelDistribution(testing_images_pytorch,testing_labels_pytorch,2,0.1)
detection_dataset = TensorDataset(detection_images,detection_labels)
detection_loader  = DataLoader(detection_dataset, batch_size=1, shuffle=True, pin_memory=True,drop_last=True)


In [ ]:
r_null   = FilterAlgorithm.runDetector(model,device,lambda_vals_training,testing_loader,0.01)
r_detect = FilterAlgorithm.runDetector(model,device,lambda_vals_training,detection_loader,0.01)

# Plot detection signal

In [ ]:
import PlottingFigures
import matplotlib.pyplot as plt

PlottingFigures.timeSeries(r_null,r_detect)

In [ ]:
PlottingFigures.plotHistogram(r_null,r_detect,15)